In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
import keras
from keras.datasets import cifar10

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
LATENT_DIMS = [256]
latent_dim = 256
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 3
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 512
ORIGINAL_DIM = 784
results_path = 'results\\cifar10_bigan_dbal'
epsilon_std = 1.0

if not os.path.exists(results_path):
    os.makedirs(results_path)

ACQ_FUNCS = {
    "var_ratios": var_ratios,
    "mean_std": mean_std,
    "max_entropy": max_entropy,
    "bald": bald,
    "uniform": uniform,
#     "batch_bald": batch_bald
}


In [4]:
class MLP_REG(nn.Module):
    def __init__(self, latent_dim):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = (X_train.astype(np.float32)) / 255.
X_train = X_train.reshape(X_train.shape[0], -1)

X_test = (X_test.astype(np.float32)) / 255.
X_test = X_test.reshape(X_test.shape[0], -1)

In [6]:
y_train = y_train.reshape((y_train.shape[0], ))
y_test = y_test.reshape((y_test.shape[0], ))

In [7]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=98,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [8]:
X_train_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\cifar10\x_train_enhanced_bigan.npy")
X_test_enhanced = np.load(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\cifar10\x_test_enhanced_bigan.npy")

for exp_iter in range(EXPERIMENT_COUNT):
    np.random.seed(exp_iter)
    initial_idx = np.array([],dtype=int)
    for i in range(10):
        idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
        initial_idx = np.concatenate((initial_idx, idx))

    for func_name, acquisition_func in ACQ_FUNCS.items():  

        X_initial = X_train_enhanced[initial_idx]
        y_initial = y_train[initial_idx]

        X_pool = np.delete(X_train_enhanced, initial_idx, axis=0)
        y_pool = np.delete(y_train, initial_idx, axis=0)

        model = MLP_REG(latent_dim).to(DEVICE)

        estimator = NeuralNetClassifier(model,
                                      max_epochs=MAX_EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      lr=LEARNING_RATE,
                                      optimizer=torch.optim.Adam,
                                      criterion=torch.nn.CrossEntropyLoss,
                                      train_split=None,
                                      verbose=0,
                                      device=DEVICE)



        acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                          X_test_enhanced,
                                                          y_test,
                                                          X_pool,
                                                          y_pool,
                                                          X_initial,
                                                          y_initial,
                                                          estimator,)
        file_name = os.path.join(results_path, "{func_name}_latent_dim_{latent_dim}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter, latent_dim=latent_dim))
        np.save(file_name, (acc_arr, dataset_size_arr))


Accuracy after query 1: 0.1450
Accuracy after query 2: 0.1605
Accuracy after query 3: 0.1703
Accuracy after query 4: 0.1879
Accuracy after query 5: 0.1965
Accuracy after query 6: 0.1936
Accuracy after query 7: 0.1969
Accuracy after query 8: 0.2074
Accuracy after query 9: 0.2048
Accuracy after query 10: 0.2225
Accuracy after query 11: 0.2108
Accuracy after query 12: 0.2157
Accuracy after query 13: 0.2215
Accuracy after query 14: 0.2189
Accuracy after query 15: 0.2237
Accuracy after query 16: 0.2260
Accuracy after query 17: 0.2265
Accuracy after query 18: 0.2279
Accuracy after query 19: 0.2456
Accuracy after query 20: 0.2457
Accuracy after query 21: 0.2446
Accuracy after query 22: 0.2522
Accuracy after query 23: 0.2522
Accuracy after query 24: 0.2497
Accuracy after query 25: 0.2509
Accuracy after query 26: 0.2537
Accuracy after query 27: 0.2565
Accuracy after query 28: 0.2510
Accuracy after query 29: 0.2568
Accuracy after query 30: 0.2596
Accuracy after query 31: 0.2596
Accuracy after qu

Accuracy after query 61: 0.2446
Accuracy after query 62: 0.2464
Accuracy after query 63: 0.2459
Accuracy after query 64: 0.2526
Accuracy after query 65: 0.2456
Accuracy after query 66: 0.2509
Accuracy after query 67: 0.2477
Accuracy after query 68: 0.2491
Accuracy after query 69: 0.2540
Accuracy after query 70: 0.2541
Accuracy after query 71: 0.2453
Accuracy after query 72: 0.2553
Accuracy after query 73: 0.2579
Accuracy after query 74: 0.2510
Accuracy after query 75: 0.2560
Accuracy after query 76: 0.2560
Accuracy after query 77: 0.2541
Accuracy after query 78: 0.2520
Accuracy after query 79: 0.2570
Accuracy after query 80: 0.2515
Accuracy after query 81: 0.2545
Accuracy after query 82: 0.2565
Accuracy after query 83: 0.2568
Accuracy after query 84: 0.2627
Accuracy after query 85: 0.2578
Accuracy after query 86: 0.2570
Accuracy after query 87: 0.2572
Accuracy after query 88: 0.2532
Accuracy after query 89: 0.2577
Accuracy after query 90: 0.2603
Accuracy after query 91: 0.2621
Accuracy

Accuracy after query 23: 0.2461
Accuracy after query 24: 0.2360
Accuracy after query 25: 0.2554
Accuracy after query 26: 0.2537
Accuracy after query 27: 0.2526
Accuracy after query 28: 0.2576
Accuracy after query 29: 0.2604
Accuracy after query 30: 0.2627
Accuracy after query 31: 0.2556
Accuracy after query 32: 0.2632
Accuracy after query 33: 0.2609
Accuracy after query 34: 0.2606
Accuracy after query 35: 0.2653
Accuracy after query 36: 0.2626
Accuracy after query 37: 0.2653
Accuracy after query 38: 0.2661
Accuracy after query 39: 0.2701
Accuracy after query 40: 0.2676
Accuracy after query 41: 0.2685
Accuracy after query 42: 0.2708
Accuracy after query 43: 0.2687
Accuracy after query 44: 0.2694
Accuracy after query 45: 0.2704
Accuracy after query 46: 0.2700
Accuracy after query 47: 0.2800
Accuracy after query 48: 0.2793
Accuracy after query 49: 0.2792
Accuracy after query 50: 0.2795
Accuracy after query 51: 0.2775
Accuracy after query 52: 0.2847
Accuracy after query 53: 0.2799
Accuracy

Accuracy after query 83: 0.2705
Accuracy after query 84: 0.2736
Accuracy after query 85: 0.2680
Accuracy after query 86: 0.2766
Accuracy after query 87: 0.2753
Accuracy after query 88: 0.2730
Accuracy after query 89: 0.2756
Accuracy after query 90: 0.2745
Accuracy after query 91: 0.2719
Accuracy after query 92: 0.2795
Accuracy after query 93: 0.2759
Accuracy after query 94: 0.2734
Accuracy after query 95: 0.2727
Accuracy after query 96: 0.2784
Accuracy after query 97: 0.2811
Accuracy after query 98: 0.2815
Accuracy after query 1: 0.1598
Accuracy after query 2: 0.1704
Accuracy after query 3: 0.1664
Accuracy after query 4: 0.1842
Accuracy after query 5: 0.1981
Accuracy after query 6: 0.2181
Accuracy after query 7: 0.2120
Accuracy after query 8: 0.2172
Accuracy after query 9: 0.2215
Accuracy after query 10: 0.2260
Accuracy after query 11: 0.2299
Accuracy after query 12: 0.2315
Accuracy after query 13: 0.2378
Accuracy after query 14: 0.2404
Accuracy after query 15: 0.2421
Accuracy after qu

Accuracy after query 45: 0.2685
Accuracy after query 46: 0.2743
Accuracy after query 47: 0.2704
Accuracy after query 48: 0.2677
Accuracy after query 49: 0.2721
Accuracy after query 50: 0.2749
Accuracy after query 51: 0.2775
Accuracy after query 52: 0.2884
Accuracy after query 53: 0.2828
Accuracy after query 54: 0.2768
Accuracy after query 55: 0.2795
Accuracy after query 56: 0.2793
Accuracy after query 57: 0.2832
Accuracy after query 58: 0.2867
Accuracy after query 59: 0.2821
Accuracy after query 60: 0.2827
Accuracy after query 61: 0.2818
Accuracy after query 62: 0.2843
Accuracy after query 63: 0.2868
Accuracy after query 64: 0.2846
Accuracy after query 65: 0.2875
Accuracy after query 66: 0.2844
Accuracy after query 67: 0.2868
Accuracy after query 68: 0.2962
Accuracy after query 69: 0.2885
Accuracy after query 70: 0.2941
Accuracy after query 71: 0.2862
Accuracy after query 72: 0.2866
Accuracy after query 73: 0.2903
Accuracy after query 74: 0.2874
Accuracy after query 75: 0.2859
Accuracy

Accuracy after query 7: 0.2136
Accuracy after query 8: 0.2149
Accuracy after query 9: 0.2157
Accuracy after query 10: 0.2222
Accuracy after query 11: 0.2284
Accuracy after query 12: 0.2306
Accuracy after query 13: 0.2306
Accuracy after query 14: 0.2364
Accuracy after query 15: 0.2443
Accuracy after query 16: 0.2459
Accuracy after query 17: 0.2526
Accuracy after query 18: 0.2418
Accuracy after query 19: 0.2564
Accuracy after query 20: 0.2601
Accuracy after query 21: 0.2542
Accuracy after query 22: 0.2545
Accuracy after query 23: 0.2523
Accuracy after query 24: 0.2541
Accuracy after query 25: 0.2559
Accuracy after query 26: 0.2545
Accuracy after query 27: 0.2541
Accuracy after query 28: 0.2552
Accuracy after query 29: 0.2578
Accuracy after query 30: 0.2621
Accuracy after query 31: 0.2591
Accuracy after query 32: 0.2632
Accuracy after query 33: 0.2561
Accuracy after query 34: 0.2593
Accuracy after query 35: 0.2549
Accuracy after query 36: 0.2553
Accuracy after query 37: 0.2653
Accuracy af